# Introdution to Artificial Neural Networks with Keras

## The Perceptron
**Forward step** is $h_{\mathbf{W}, \mathbf{b}} = \phi(\mathbf{X}\mathbf{W} + \mathbf{b})$ (assuming the addition is broadcasted).

**Learning step** is $w_{i,j}^{(\text{next})} = w_{i,j} + \eta (y_j - \hat{y}_j) x_i$ where:
* $x_i$ is the $i$-th input
* $y_j$ is the $j$-th target
* $\hat{y}_j$ is the $j$-th predicted value
* $\eta$ is the learning rate

The learning is basically a gradient descent with constant learning rate and specific gradient.

It can be shown that for linearly separable dataset this algorightm converges and finds a separating hyper-plane.

In [1]:
import numpy as np

from sklearn.datasets import load_iris
from sklearn.linear_model import Perceptron

# Load the IRIS dataset
iris = load_iris()
X = iris.data[:, (2, 3)]  # petal length, petal width
y = (iris.target == 0).astype(np.int)

# Train Perceptron with single unit
per_clf = Perceptron(max_iter=1000, tol=1e-3, random_state=42)
per_clf.fit(X, y)

# Make a prediction
per_clf.predict([[2, 0.5]])

array([1])